In [1]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from utils.postgres import initialize_db

Added finer to sys.path


In [2]:
initialize_db()